# 一、爬取目标入口站点

In [ ]:
import requests
import re

G_TARGET_URL = 'https://store.steampowered.com/search/?category1=998&filter=topsellers'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36'
}
response = requests.get(G_TARGET_URL, headers=headers)
content = response.text

## 测试匹配 url 的正则表达式

In [ ]:
pattern = re.compile('<a href="([a-zA-z]+://[^\s]*?)".*?data-ds-appid=.*?data-ds-tagids=.*?',re.S)
results = re.findall(pattern, content)
#results

## 爬取总页数并收集每个游戏的 url 链接

- 总页数实时变化
- 爬取的详细链接都放入 GAME_URL 该链表中

In [ ]:
page_compile = re.compile('page=(.*?)" onclick=',re.S)
content = response.text
pages = re.findall(page_compile, content)
max_page = pages[2]
print('total pages is :',max_page)
GAME_URL = []
pattern = re.compile('<a href="([a-zA-z]+://[^\s]*?)".*?data-ds-appid',re.S)
for page in range(int(max_page)+1):
    targe_url = G_TARGET_URL+'&page='+str(page)
    try :
        response = requests.get(targe_url, headers=headers)
        content = response.text
        results = re.findall(pattern, content)
        for url in results:
            if len(url)< 60 :
                #print("del wrong url :",url)
                results.remove(url)
                
        GAME_URL.extend(results)
        print(targe_url," is OK"+'\r',end="")
    except:
        print(targe_url," is NOT OK"+'\r',end="")

## 查看所有详细链接

In [ ]:
GAME_URL

## 所有详细链接保存至 urls.txt

In [ ]:
with open("urls.txt",'w+') as f:
    for url in GAME_URL:
        f.write(str(url))
        f.write('\n')

# 二、爬取单个游戏的详细信息

In [ ]:
TARGET_URL = 'https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/'
response = requests.get(TARGET_URL, headers=headers)
content = response.text


## 获取游戏名

In [ ]:
name_pattern = re.compile('<div class="apphub_AppName">(.*?)</div>',re.S)
results = re.findall(name_pattern, content)
results

## 获取游戏公司

In [ ]:
developer_pattern = re.compile(r'"developers_list".*?<a href=.*?>(.*?)</a>',re.S)
results = re.findall(developer_pattern, content)
results

## 获取好评等级

In [ ]:
description_pattern = re.compile('<span class="game_review_summary positive" itemprop="description">(.*?)</span>',re.S)
results = re.findall(description_pattern, content)
results

## 获取评价人数

In [ ]:
responsive_pattern = re.compile('<span class="responsive_hidden">.*?\\((.*?)\\).*?</span>',re.S)
results = re.findall(responsive_pattern, content)
results[1]

## 获取发行时间

In [ ]:
date_pattern = re.compile('<div class="date">(.*?)</div>',re.S)
results = re.findall(date_pattern, content)
results

## 获取价格

In [ ]:
price_pattern = re.compile('<div class="game_purchase_price price".*?>(.*?)</div>',re.S)
price = re.findall(price_pattern, content)
if len(price) <1:
    bs = etree.HTML(content) 
    price = bs.xpath('//div [@class="discount_prices"]/div[2]')[0].text
price[0].strip()

## 获取系统需求

In [ ]:
from lxml import etree 
bs = etree.HTML(content) 
content_active = bs.xpath('//div [@class = "game_area_sys_req sysreq_content active"]/div[1]/ul/ul/li')
results= []
for i in content_active:
    results.append(i.xpath('./text()'))
results

## 获取游戏标签

In [ ]:
table_pattern = re.compile('<a href="https://store.steampowered.com/tags/.*?" class="app_tag" style=.*?>(.*?)</a>',re.S)
results = re.findall(table_pattern, content)
new_res = []
for r in results:
    new_res.append(r.strip())
new_res

## 获取语言种类

In [ ]:
language_pattern = re.compile('<td style=.*? class="ellipsis">(.*?)</td>',re.S)
results = re.findall(language_pattern, content)
new_res = []
for r in results:
    new_res.append(r.strip())
new_res 

# 三、爬取所有游戏的详细信息并存成 csv 文件 

- 2000 多页大约耗时 25 分钟

In [ ]:
NAME = []
COMPANY = []
PRAISE_LEVEL=[]
COMMENT_NUMBER = []
LANGUAGE = []
PRICE = []
ISSUE_TIME = []
SYSTEM_REQ = []
GGME_TAG = []
index = 0
session = requests.Session()
for url in GAME_URL:
#     index = index+1  #调试 10 页
#     if index>10:
#         break
    session.cookies['cookie'] = 'wants_mature_content=1; browserid=1340293676165560377; sessionid=e882a8713f7f85abfeb4452e; timezoneOffset=28800,0; _ga=GA1.2.2034098390.1556032675; _gid=GA1.2.782308328.1556032675; Steam_Language=english; app_impressions=10:80@1_5_9__412|300@1_5_9__412|20@1_5_9__412|30@1_5_9__412|40@1_5_9__412|50@1_5_9__412|60@1_5_9__412|70@1_5_9__412|130@1_5_9__412|291480@1_5_9__300|222880@1_5_9__300|10@1_5_9__300|240@1_5_9__300|10:80@1_5_9__412|300@1_5_9__412|20@1_5_9__412|30@1_5_9__412|40@1_5_9__412|50@1_5_9__412|60@1_5_9__412|70@1_5_9__412|130@1_5_9__412|291480@1_5_9__300|222880@1_5_9__300|10@1_5_9__300|240@1_5_9__300; steamCountry=CN%7Ce2a2b396cdbf0bada4bc1dd9a31f10fe; birthtime=723139201; lastagecheckage=1-0-1993; recentapps=%7B%22239140%22%3A1556039491%2C%22570%22%3A1556039076%2C%22730%22%3A1556034113%7D'
    response = session.get(url, headers=headers)
    content = response.text
    try:
        #游戏名
        name_pattern = re.compile('<div class="apphub_AppName">(.*?)</div>',re.S)
        name = re.findall(name_pattern, content)
        print("name is OK !")
        #公司
        developer_pattern = re.compile(r'"developers_list".*?<a href=.*?>(.*?)</a>',re.S)
        developer = re.findall(developer_pattern, content) 
        print("developer is OK !")
        #评论人数
        responsive_pattern = re.compile('<span class="responsive_hidden">.*?\\((.*?)\\).*?</span>',re.S)
        comment_num = re.findall(responsive_pattern, content)
        print("comment_num is OK !")
        #好评等级
        description_pattern = re.compile('<span class="game_review_summary positive" itemprop="description">(.*?)</span>',re.S)
        description = re.findall(description_pattern, content)
        print("description is OK !")
        #语言种类
        language_pattern = re.compile('<td style=.*? class="ellipsis">(.*?)</td>',re.S)
        languages = re.findall(language_pattern, content)
        language_res = []
        for r in languages:
            language_res.append(r.strip())
        print("languages is OK !")
        #价格
        price_pattern = re.compile('<div class="game_purchase_price price".*?>(.*?)</div>',re.S)
        price = re.findall(price_pattern, content)
        #打折情况
        if len(price) <1: 
            bs = etree.HTML(content) 
            price = bs.xpath('//div [@class="discount_prices"]/div[2]')[0].text
        print("price is OK !")
        #发行日期
        date_pattern = re.compile('<div class="date">(.*?)</div>',re.S)
        date = re.findall(date_pattern, content)
        print("date is OK !")
        #配置要求
        bs = etree.HTML(content) 
        content_active = bs.xpath('//div [@class = "game_area_sys_req sysreq_content active"]/div[1]/ul/ul/li')
        req= []
        for i in content_active:
            req.append(i.xpath('./text()'))
        print("req is OK !")
        #游戏标签
        tag_pattern = re.compile('<a href="https://store.steampowered.com/tags/.*?" class="app_tag" style=.*?>(.*?)</a>',re.S)
        tags = re.findall(tag_pattern, content)
        tag_res = []
        for r in tags:
            new_res.append(r.strip())     
        print("tags is OK !")
        
        NAME.append(name)
        COMPANY.append(developer)
        PRAISE_LEVEL.append(description)
        if len(comment_num)>=2:
            COMMENT_NUMBER.append(comment_num[1])
        else:
            COMMENT_NUMBER.append('No user reviews')
        LANGUAGE.append(language_res)
        PRICE.append(price[0].strip())
        ISSUE_TIME.append(date)
        SYSTEM_REQ.append(req)
        GGME_TAG.append(tag_res)
        print(url," is OK !")
        
    except:
        #有些网页需要登录账号查看，所以会导致获取数据失败
        print(url," is Not OK !!!!!!!!!!!!!!!!!!!!!!!!!!")

In [ ]:
import pandas as pd
from pandas.core.frame import DataFrame
 
all_data={"name" : NAME,
   "company" : COMPANY,
   "praise level" : PRAISE_LEVEL,
   "comment number" : COMMENT_NUMBER,
   "language" : LANGUAGE,
   "price" : PRICE,
   "issue time" : ISSUE_TIME,
   "system require" : SYSTEM_REQ,
   "game tags" : GGME_TAG}#将列表转换成字典
data=DataFrame(all_data)#将字典转换成为数据框

In [ ]:
# 简单清洗数据
data_copy = data
def dataFilter(category):
    s = str(category)
    s = s.replace('[','')
    s = s.replace(']','')
    s = s.replace('\'','')
    s = s.replace('¥','')
    return s

data['Name'] = data_copy['name'].apply(dataFilter)
data['Company'] = data_copy['company'].apply(dataFilter)
data['Praise Level'] = data_copy['praise level'].apply(dataFilter)
data['Comment Number'] = data_copy['comment number'].apply(dataFilter)
data['Language'] = data_copy['language'].apply(dataFilter)
data['Price'] = data_copy['price'].apply(dataFilter)
data['Issue time'] = data_copy['issue time'].apply(dataFilter)
data['System Require'] = data_copy['system require'].apply(dataFilter)
data['Game Tags'] = data_copy['game tags'].apply(dataFilter)

del data['name']
del data['company']
del data['praise level']
del data['comment number']
del data['language']
del data['price']
del data['issue time']
del data['system require']
del data['game tags']

In [ ]:
# encoding 防止中文乱码
data.to_csv('all_data.csv',encoding='utf_8_sig')